In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /Users/jiwonmoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Read data files_Comments, Customers
comment = pd.read_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 4/Comments.csv')
display(comment)

customer = pd.read_csv('/Users/jiwonmoon/Desktop/CIS 508/Data Mining I/Assignments/Individual Assignment 4/Customers.csv')
display(customer)

print(comment.shape)
print(customer.shape)

,ID,Comments
0,1309,Does not like the way the phone works. It is t...
1,3556,Wanted to know the nearest store location. Wan...
2,2230,Wants to know how to do text messaging. Referr...
3,2312,Asked how to disable call waiting. referred hi...
4,3327,Needs help learning how to use the phone. I su...
...,...,...
2065,3034,Needed help figuring out his bill. I explained...
2066,271,He lost his phone and called to cancel service...
2067,783,Lost the directions to phone and wants another...
2068,1295,Wants to change address.


,ID,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,TARGET
0,1,F,S,1,38000.00,N,229.64,24.393333,3,23.56,0.00,206.08,0,CC,Budget,Intnl_discount,Cancelled
1,6,M,M,2,29616.00,N,75.29,49.426667,2,29.78,0.00,45.50,0,CH,FreeLocal,Standard,Current
2,8,M,M,0,19732.80,N,47.25,50.673333,3,24.81,0.00,22.44,0,CC,FreeLocal,Standard,Current
3,11,M,S,2,96.33,N,59.01,56.473333,1,26.13,0.00,32.88,1,CC,Budget,Standard,Current
4,14,F,M,2,52004.80,N,28.14,25.140000,1,5.03,0.00,23.11,0,CH,Budget,Intnl_discount,Cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,F,S,0,78851.30,N,29.04,48.373333,4,0.37,0.00,28.66,0,CC,FreeLocal,Standard,Cancelled
2066,3822,F,S,1,17540.70,Y,36.20,62.786667,1,22.17,0.57,13.45,0,Auto,Budget,Standard,Cancelled
2067,3823,F,M,0,83891.90,Y,74.40,61.020000,4,28.92,0.00,45.47,0,CH,Budget,Standard,Cancelled
2068,3824,F,M,2,28220.80,N,38.95,38.766667,4,26.49,0.00,12.46,0,CC,FreeLocal,Standard,Cancelled


(2070, 2)
(2070, 17)


In [3]:
# Extract target column from Customers
y_train = customer["TARGET"]
X_train = customer.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(comment.shape)
print(comment.head())
print(y_train)

(2070, 16)
(2070, 2)
     ID                                           Comments
0  1309  Does not like the way the phone works. It is t...
1  3556  Wanted to know the nearest store location. Wan...
2  2230  Wants to know how to do text messaging. Referr...
3  2312  Asked how to disable call waiting. referred hi...
4  3327  Needs help learning how to use the phone. I su...
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


## 1. Stemming - Snowball, Porter

In [4]:
# Tokenize - Split the sentences to lists of words
comment['CommentsTokenized'] = comment['Comments'].apply(word_tokenize)

### 1-1. Snowball

In [5]:
# Snowball Stemmer
snowball = SnowballStemmer(language='english')

# Now do stemming - create a new dataframe to store stemmed version
S_text = pd.DataFrame()
S_text = comment.drop(columns = ['CommentsTokenized', 'Comments'])
S_text['CommentsTokenizedStemmed'] = comment['CommentsTokenized'].apply(lambda x: [snowball.stem(y) for y in x])

In [6]:
S_text

,ID,CommentsTokenizedStemmed
0,1309,"[doe, not, like, the, way, the, phone, work, ...."
1,3556,"[want, to, know, the, nearest, store, locat, ...."
2,2230,"[want, to, know, how, to, do, text, messag, .,..."
3,2312,"[ask, how, to, disabl, call, wait, ., refer, h..."
4,3327,"[need, help, learn, how, to, use, the, phone, ..."
...,...,...
2065,3034,"[need, help, figur, out, his, bill, ., i, expl..."
2066,271,"[he, lost, his, phone, and, call, to, cancel, ..."
2067,783,"[lost, the, direct, to, phone, and, want, anot..."
2068,1295,"[want, to, chang, address, .]"


In [7]:
# Join stemmed strings
S_text['CommentsTokenizedStemmed'] = S_text['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

In [8]:
S_text

,ID,CommentsTokenizedStemmed
0,1309,doe not like the way the phone work . it is to...
1,3556,want to know the nearest store locat . want to...
2,2230,want to know how to do text messag . refer him...
3,2312,ask how to disabl call wait . refer him to web...
4,3327,need help learn how to use the phone . i sugge...
...,...,...
2065,3034,need help figur out his bill . i explain our m...
2066,271,he lost his phone and call to cancel servic . ...
2067,783,lost the direct to phone and want anoth manual...
2068,1295,want to chang address .


### 1-2. Porter

In [9]:
# Use Porter Stemmer
porter = PorterStemmer()

#Now do stemming - create a new dataframe to store stemmed version
P_text = pd.DataFrame()
P_text = comment.drop(columns = ['CommentsTokenized', 'Comments'])
P_text['CommentsTokenizedStemmed'] = comment['CommentsTokenized'].apply(lambda x: [porter.stem(y) for y in x])

In [10]:
P_text

,ID,CommentsTokenizedStemmed
0,1309,"[doe, not, like, the, way, the, phone, work, ...."
1,3556,"[want, to, know, the, nearest, store, locat, ...."
2,2230,"[want, to, know, how, to, do, text, messag, .,..."
3,2312,"[ask, how, to, disabl, call, wait, ., refer, h..."
4,3327,"[need, help, learn, how, to, use, the, phone, ..."
...,...,...
2065,3034,"[need, help, figur, out, hi, bill, ., i, expla..."
2066,271,"[he, lost, hi, phone, and, call, to, cancel, s..."
2067,783,"[lost, the, direct, to, phone, and, want, anot..."
2068,1295,"[want, to, chang, address, .]"


In [11]:
# Join stemmed strings
P_text['CommentsTokenizedStemmed'] = P_text['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

In [12]:
P_text

,ID,CommentsTokenizedStemmed
0,1309,doe not like the way the phone work . it is to...
1,3556,want to know the nearest store locat . want to...
2,2230,want to know how to do text messag . refer him...
3,2312,ask how to disabl call wait . refer him to web...
4,3327,need help learn how to use the phone . i sugge...
...,...,...
2065,3034,need help figur out hi bill . i explain our mi...
2066,271,he lost hi phone and call to cancel servic . i...
2067,783,lost the direct to phone and want anoth manual...
2068,1295,want to chang address .


#### Choosing Snowball because it is better stemmed than Porter (i.e. 2066, ID 271 - Snowball:his / Porter:hi)

## 2. TF-IDF

In [13]:
# Eliminating stop word
count_vect = CountVectorizer(stop_words = None)
count_vect = CountVectorizer(stop_words = 'english',lowercase = False)

In [14]:
# Term-Document Matrix
TD_counts = count_vect.fit_transform(S_text.CommentsTokenizedStemmed)
print(TD_counts.shape)
print(TD_counts.dtype)
print(count_vect.get_feature_names())
print(TD_counts)
DF_TD_Counts = pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)

(2070, 354)
int64
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effe

In [15]:
# Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_train_tfidf.shape)

TF_IDF = pd.DataFrame(X_train_tfidf.toarray())
print(TF_IDF)

(2070, 354)
      0    1    2    3        4    5    6    7    8         9    ...  344  \
0     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
1     0.0  0.0  0.0  0.0  0.27568  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
3     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
4     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
...   ...  ...  ...  ...      ...  ...  ...  ...  ...       ...  ...  ...   
2065  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2066  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2067  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2068  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.772949  ...  0.0   
2069  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   

      345  346       347  348  349  350  351  352  353  
0     

## 3. One Hot Encoding

In [16]:
#Merge files
print(customer.shape)
X_train = customer.drop(columns = ["TARGET"])
print(X_train.shape)

merge = pd.concat([X_train, TF_IDF], axis=1)
print(merge.shape)
print(merge)

(2070, 17)
(2070, 16)
(2070, 370)
        ID Sex Status  Children  Est_Income Car_Owner   Usage        Age  \
0        1   F      S         1    38000.00         N  229.64  24.393333   
1        6   M      M         2    29616.00         N   75.29  49.426667   
2        8   M      M         0    19732.80         N   47.25  50.673333   
3       11   M      S         2       96.33         N   59.01  56.473333   
4       14   F      M         2    52004.80         N   28.14  25.140000   
...    ...  ..    ...       ...         ...       ...     ...        ...   
2065  3821   F      S         0    78851.30         N   29.04  48.373333   
2066  3822   F      S         1    17540.70         Y   36.20  62.786667   
2067  3823   F      M         0    83891.90         Y   74.40  61.020000   
2068  3824   F      M         2    28220.80         N   38.95  38.766667   
2069  3825   F      S         0    28589.10         N  100.28  15.600000   

      RatePlan  LongDistance  ...  344  345  346     

In [17]:
print (merge.dtypes)
merge.select_dtypes('object').head()

ID              int64
Sex            object
Status         object
Children        int64
Est_Income    float64
               ...   
349           float64
350           float64
351           float64
352           float64
353           float64
Length: 370, dtype: object


,Sex,Status,Car_Owner,Paymethod,LocalBilltype,LongDistanceBilltype
0,F,S,N,CC,Budget,Intnl_discount
1,M,M,N,CH,FreeLocal,Standard
2,M,M,N,CC,FreeLocal,Standard
3,M,S,N,CC,Budget,Standard
4,F,M,N,CH,Budget,Intnl_discount


In [18]:
# Do one Hot encoding for categorical features
categorical = ['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
print(categorical)

comment_f = pd.get_dummies(merge,columns = categorical)
print(comment_f.shape)

['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 378)


In [19]:
comment_f

,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,...,Status_S,Car_Owner_N,Car_Owner_Y,Paymethod_Auto,Paymethod_CC,Paymethod_CH,LocalBilltype_Budget,LocalBilltype_FreeLocal,LongDistanceBilltype_Intnl_discount,LongDistanceBilltype_Standard
0,1,1,38000.00,229.64,24.393333,3,23.56,0.00,206.08,0,...,1,1,0,0,1,0,1,0,1,0
1,6,2,29616.00,75.29,49.426667,2,29.78,0.00,45.50,0,...,0,1,0,0,0,1,0,1,0,1
2,8,0,19732.80,47.25,50.673333,3,24.81,0.00,22.44,0,...,0,1,0,0,1,0,0,1,0,1
3,11,2,96.33,59.01,56.473333,1,26.13,0.00,32.88,1,...,1,1,0,0,1,0,1,0,0,1
4,14,2,52004.80,28.14,25.140000,1,5.03,0.00,23.11,0,...,0,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,0,78851.30,29.04,48.373333,4,0.37,0.00,28.66,0,...,1,1,0,0,1,0,0,1,0,1
2066,3822,1,17540.70,36.20,62.786667,1,22.17,0.57,13.45,0,...,1,0,1,1,0,0,1,0,0,1
2067,3823,0,83891.90,74.40,61.020000,4,28.92,0.00,45.47,0,...,0,0,1,0,0,1,1,0,0,1
2068,3824,2,28220.80,38.95,38.766667,4,26.49,0.00,12.46,0,...,0,1,0,0,1,0,0,1,0,1


## 4. Feature Selection_Filter, Wrapper

### 4-1. Filter Top 50

In [20]:
selector = SelectKBest(score_func = chi2, k = 50)

Filter_TF_IDF_50 = selector.fit_transform(TF_IDF, y_train)
print(Filter_TF_IDF_50.shape)

feature_names_out = selector.get_support(indices=True)
print(feature_names_out)

TF_IDF_Filter= pd.DataFrame(Filter_TF_IDF_50)
print(TF_IDF_Filter)

(2070, 50)
[  9  11  14  26  49  50  51  60  62  70  81  84  94 115 118 121 144 157
 168 170 172 183 186 190 194 205 212 213 217 222 234 235 239 244 245 248
 249 256 259 266 273 296 307 313 315 318 319 323 338 342]
            0    1    2    3         4         5    6         7    8    9   \
0     0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.465653  0.0  0.0   
1     0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
2     0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
3     0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
4     0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
...        ...  ...  ...  ...       ...       ...  ...       ...  ...  ...   
2065  0.000000  0.0  0.0  0.0  0.000000  0.446161  0.0  0.000000  0.0  0.0   
2066  0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
2067  0.000000  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
2068 

In [21]:
# train_test_split
y = customer["TARGET"]
x_train, x_test, Y_train, Y_test = train_test_split(TF_IDF_Filter, y, test_size = 0.2, random_state = 42)

In [22]:
rf = RandomForestClassifier()

rf.fit(x_train, Y_train)
preds = rf.predict(x_test)

score = rf.score(x_test, Y_test)
print(score)
print('Classification Report :')
print(classification_report(Y_test, preds))

0.6280193236714976
Classification Report :
              precision    recall  f1-score   support

   Cancelled       0.56      0.10      0.16       157
     Current       0.63      0.95      0.76       257

    accuracy                           0.63       414
   macro avg       0.59      0.52      0.46       414
weighted avg       0.60      0.63      0.53       414



### 4-2. Filter Top 25

In [30]:
selector = SelectKBest(score_func = chi2, k = 25)

Filter_TF_IDF_25 = selector.fit_transform(TF_IDF, y_train)
print(Filter_TF_IDF_25.shape)

feature_names_out = selector.get_support(indices=True)
print(feature_names_out)

TF_IDF_Filter= pd.DataFrame(Filter_TF_IDF_25)
print(TF_IDF_Filter)

(2070, 25)
[ 11  14  49  50  70  81 115 118 121 144 186 190 194 212 217 222 239 244
 248 249 259 273 313 319 323]
       0    1         2         3    4    5         6    7         8    9   \
0     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
1     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
3     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
4     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
...   ...  ...       ...       ...  ...  ...       ...  ...       ...  ...   
2065  0.0  0.0  0.000000  0.446161  0.0  0.0  0.460113  0.0  0.457852  0.0   
2066  0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2067  0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2068  0.0  0.0  0.545354  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2069  0.0  0.0  0.000000  0.

In [31]:
# train_test_split
y = customer["TARGET"]
x_train, x_test, Y_train, Y_test = train_test_split(TF_IDF_Filter, y, test_size = 0.2, random_state = 42)

In [32]:
rf = RandomForestClassifier()

rf.fit(x_train, Y_train)
preds = rf.predict(x_test)

score = rf.score(x_test, Y_test)
print(score)
print('Classification Report :')
print(classification_report(Y_test, preds))

0.6280193236714976
Classification Report :
              precision    recall  f1-score   support

   Cancelled       0.57      0.08      0.13       157
     Current       0.63      0.96      0.76       257

    accuracy                           0.63       414
   macro avg       0.60      0.52      0.45       414
weighted avg       0.61      0.63      0.52       414



### 4-3. Filter Top 10

In [33]:
selector = SelectKBest(score_func = chi2, k = 25)

Filter_TF_IDF_10 = selector.fit_transform(TF_IDF, y_train)
print(Filter_TF_IDF_10.shape)

feature_names_out = selector.get_support(indices=True)
print(feature_names_out)

TF_IDF_Filter= pd.DataFrame(Filter_TF_IDF_10)
print(TF_IDF_Filter)

(2070, 25)
[ 11  14  49  50  70  81 115 118 121 144 186 190 194 212 217 222 239 244
 248 249 259 273 313 319 323]
       0    1         2         3    4    5         6    7         8    9   \
0     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
1     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
3     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
4     0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
...   ...  ...       ...       ...  ...  ...       ...  ...       ...  ...   
2065  0.0  0.0  0.000000  0.446161  0.0  0.0  0.460113  0.0  0.457852  0.0   
2066  0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2067  0.0  0.0  0.000000  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2068  0.0  0.0  0.545354  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.0   
2069  0.0  0.0  0.000000  0.

In [34]:
# train_test_split
y = customer["TARGET"]
x_train, x_test, Y_train, Y_test = train_test_split(TF_IDF_Filter, y, test_size = 0.2, random_state = 42)

In [35]:
rf = RandomForestClassifier()

rf.fit(x_train, Y_train)
preds = rf.predict(x_test)

score = rf.score(x_test, Y_test)
print(score)
print('Classification Report :')
print(classification_report(Y_test, preds))

0.6207729468599034
Classification Report :
              precision    recall  f1-score   support

   Cancelled       0.50      0.06      0.10       157
     Current       0.63      0.96      0.76       257

    accuracy                           0.62       414
   macro avg       0.56      0.51      0.43       414
weighted avg       0.58      0.62      0.51       414



### 4-4-1. Wrapper Top 50_RF

In [24]:
# RandomForestClassifier
rfc = RandomForestClassifier()
selector = SelectFromModel(rfc, max_features = 50, threshold = -np.inf)

TF_IDF_RF = selector.fit_transform(TF_IDF, y_train)
print(TF_IDF_RF.shape)

feature_names_out = selector.get_support(indices = True)
print(feature_names_out)

TF_IDF_Wrapper_RF = pd.DataFrame(TF_IDF_RF)
print(TF_IDF_Wrapper_RF)

(2070, 50)
[  7   9  14  22  23  27  43  49  50  55  70  83  91  98 106 115 118 121
 135 141 144 157 161 164 171 190 194 199 201 215 220 222 229 234 237 239
 256 258 259 262 287 295 313 315 328 331 335 341 342 347]
       0         1    2    3         4    5    6         7         8    9   \
0     0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
1     0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
2     0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
3     0.0  0.000000  0.0  0.0  0.352868  0.0  0.0  0.000000  0.000000  0.0   
4     0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
...   ...       ...  ...  ...       ...  ...  ...       ...       ...  ...   
2065  0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.446161  0.0   
2066  0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
2067  0.0  0.000000  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0   
2068 

In [25]:
# train_test_split
y = customer["TARGET"]
x_train, x_test, Y_train, Y_test = train_test_split(TF_IDF_Wrapper_RF, y, test_size = 0.2, random_state = 42)

In [26]:
rf = RandomForestClassifier()

rf.fit(x_train, Y_train)
preds = rf.predict(x_test)

score = rf.score(x_test, Y_test)
print(score)
print('Classification Report :')
print(classification_report(Y_test, preds))

0.6111111111111112
Classification Report :
              precision    recall  f1-score   support

   Cancelled       0.45      0.11      0.17       157
     Current       0.63      0.92      0.75       257

    accuracy                           0.61       414
   macro avg       0.54      0.51      0.46       414
weighted avg       0.56      0.61      0.53       414



### 4-4-2. Wrapper Top 50_DT

In [27]:
# DecisionTreeClassifier
dt = DecisionTreeClassifier()
selector = SelectFromModel(dt, max_features = 50, threshold = -np.inf)

TF_IDF_DT = selector.fit_transform(TF_IDF,y_train)
print(TF_IDF_DT.shape)

feature_names_out = selector.get_support(indices=True)
print(feature_names_out)

TF_IDF_Wrapper_DT = pd.DataFrame(TF_IDF_DT)
print(TF_IDF_Wrapper_DT)

(2070, 50)
[  7   9  11  14  22  26  27  30  43  59  61  92  98 115 118 121 135 150
 155 194 199 201 209 215 220 234 236 237 239 240 244 246 256 258 262 276
 282 297 307 313 323 327 328 335 337 340 341 342 347 353]
       0         1    2    3    4    5    6    7    8    9   ...   40   41  \
0     0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1     0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2     0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3     0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4     0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...   ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2065  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2066  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2067  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2068 

In [28]:
# train_test_split
y = customer["TARGET"]
x_train, x_test, Y_train, Y_test = train_test_split(TF_IDF_Wrapper_DT, y, test_size = 0.2, random_state = 42)

In [29]:
dt = DecisionTreeClassifier()

dt.fit(x_train, Y_train)
preds = dt.predict(x_test)

score = dt.score(x_test, Y_test)
print(score)
print('Classification Report :')
print(classification_report(Y_test, preds))

0.6135265700483091
Classification Report :
              precision    recall  f1-score   support

   Cancelled       0.46      0.11      0.18       157
     Current       0.63      0.92      0.75       257

    accuracy                           0.61       414
   macro avg       0.55      0.52      0.47       414
weighted avg       0.57      0.61      0.53       414

